# Imports

In [74]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import argparse
import hypertune

In [75]:
NUM_EPOCHS = 10

In [76]:
train = pd.read_csv("train.csv")

test_incomplete = pd.read_csv("test.csv")
gender_ds = pd.read_csv("gender_submission.csv")

test = test_incomplete.merge(gender_ds, on=["PassengerId"], how="outer")

In [77]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [78]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1


# Split

In [79]:
train, val = train_test_split(train, test_size=0.2)

# Understanding data

In [80]:
train.shape

(712, 12)

In [81]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [82]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,712.000000,712.000000,712.000000,573.000000,712.000000,712.000000,712.000000
mean,451.214888,0.382022,2.301966,29.942548,0.536517,0.387640,30.987880
std,261.469121,0.486224,0.838656,14.694246,1.115705,0.829325,43.853816
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.000000,0.000000,0.000000,7.917700
50%,455.500000,0.000000,3.000000,28.000000,0.000000,0.000000,14.456250
75%,679.250000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,891.000000,1.000000,3.000000,74.000000,8.000000,6.000000,512.329200


In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 751 to 508
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          573 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Cabin        162 non-null    object 
 11  Embarked     711 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 72.3+ KB


# EDA

## Pairplot

In [84]:
# sns.pairplot(train)

## Heatmap

In [85]:
# sns.heatmap(train.corr())

# Preprocessing layers

## Hyperparameters

In [86]:
!pip -q install hypertune

In [87]:
def get_args():
  parser = argparse.ArgumentParser()
  parser.add_argument(
      "--learning_rate",
      required=True,
      type=float,
      help="learning_rate")
  parser.add_argument(
      "--num_units",
      required=True,
      type=int,
      help="number of units in last hidden layer")
  # args = parser.parse_args()
  args = parser.parse_args("")
  
  return args

## Input pipeline using tf.data

In [88]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels=dataframe.pop("Survived")
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

## Data Cleaning

In [89]:
def clean_dataset(dataset):
  dataset = dataset.drop(["Name", "Ticket", "Cabin", "Parch", "SibSp", "Embarked", "PassengerId"], axis=1)
  dataset['Age'] = dataset['Age'].fillna(dataset['Age'].median())
  return dataset

## Categorical Columns

In [90]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  if dtype == "string":
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_tokens=max_tokens)

  feature_ds = dataset.map(lambda x, y: x[name])
  index.adapt(feature_ds)
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())
  return lambda feature: encoder(index(feature))

## Normalization

In [91]:
def get_normalization_layer(name, dataset):
  normalizer = preprocessing.Normalization(axis=None)
  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)
  return normalizer

## Input pipeline

In [92]:
train = clean_dataset(train)
val = clean_dataset(val)
test = clean_dataset(test)

In [93]:
# train.info()

In [94]:
# train.isna().sum()

In [95]:
# train.head(15)

In [96]:
train_ds = df_to_dataset(train, batch_size=32)
val_ds = df_to_dataset(val, shuffle=False, batch_size=32)
test_ds = df_to_dataset(test, shuffle=False, batch_size=32)

In [97]:
all_inputs = []
encoded_features = []

for header in ["Pclass", "Age", "Fare"]:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

In [98]:
categorical_col = tf.keras.Input(shape=(1,), name="Sex", dtype='string')
encoding_layer = get_category_encoding_layer("Sex", train_ds, dtype='string', max_tokens=5)
encoded_categorical_col = encoding_layer(categorical_col)
all_inputs.append(categorical_col)
encoded_features.append(encoded_categorical_col)

# Create, compile, and train the model

In [99]:
def create_model(num_units, learning_rate):
  all_features = tf.keras.layers.concatenate(encoded_features)
  x = tf.keras.layers.Dense(32, activation="relu")(all_features)
  x = tf.keras.layers.Dense(num_units, activation="relu")(x)
  x = tf.keras.layers.Dropout(0.3)(x)
  output = tf.keras.layers.Dense(1)(x)
  model = tf.keras.Model(all_inputs, output)

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=["accuracy"])
  
  return model

In [100]:
# model = create_model(args.num_neurons, args.learning_rate)

In [101]:
# tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [102]:
# history = model.fit(train_ds, epochs=30, validation_data=val_ds)

In [103]:
# loss, accuracy = model.evaluate(test_ds)
# print("Accuracy", accuracy)

In [104]:
# model.save('my_pet_classifier')
# reloaded_model = tf.keras.models.load_model('my_pet_classifier')

In [105]:
# test.head()

In [106]:
# sample = {
#     'Pclass': 1,
#     'Sex': "female",
#     'Age': 32,
#     'Fare': 70.25,
# }

# input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
# predictions = reloaded_model.predict(input_dict)
# prob = tf.nn.sigmoid(predictions[0][0])

# print(f"This has a {prob * 100:.1f}% change of survival")

In [107]:
def main():
  args = get_args()
  model = create_model(args.num_units, args.learning_rate)
  # tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")
  history = model.fit(train_ds, epochs=NUM_EPOCHS, validation_data=val_ds)

  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=NUM_EPOCHS)

In [108]:
if __name__ == "__main__":
  main()

usage: ipykernel_launcher.py [-h] --learning_rate LEARNING_RATE --num_units
                             NUM_UNITS
ipykernel_launcher.py: error: the following arguments are required: --learning_rate, --num_units


SystemExit: 2

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
